In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [2]:
import cv2
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import models

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/

In [3]:
big_image_size = 1000
hr_size = 100
lr_size = 50

In [4]:
def big_image_to_parts(img, hr_size, big_image_size):
    r_image = cv2.resize(img, (big_image_size, big_image_size))
    height, width, channel = r_image.shape
    CROP_H_SIZE = big_image_size // hr_size
    CROP_W_SIZE = big_image_size // hr_size
    images = []
    for ih in range(CROP_H_SIZE):
        for iw in range(CROP_W_SIZE):
            x = width // CROP_W_SIZE * iw
            y = height // CROP_H_SIZE * ih
            h = (height // CROP_H_SIZE)
            w = (width // CROP_W_SIZE)
            images.append(r_image[y:y + h, x:x + w])
    return images

In [5]:
X = []
y = []
file_index = 1
for file in os.listdir("/kaggle/input/super-resolution-pics/pics"):
    file_image = cv2.imread("/kaggle/input/super-resolution-pics/pics/" + file, cv2.IMREAD_COLOR)
    pic_index = 1
    small_images = big_image_to_parts(file_image, hr_size, big_image_size)
    for img in small_images:
        pic_name = "/kaggle/input_images/y/" + str(file_index) + "_" + str(pic_index) + ".jpg"
        y.append(img)
        cv2.imwrite(pic_name, img)
        img_scale_down = cv2.resize(img, (lr_size, lr_size))
        image2 = cv2.resize(img_scale_down, (hr_size, hr_size), interpolation=cv2.INTER_LINEAR)
        X.append(image2)
        pic_name = "/kaggle/input_images/X/" + str(file_index) + "_" + str(pic_index) + ".jpg"
        cv2.imwrite(pic_name, image2)
        pic_index += 1
    file_index += 1

FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/input/super-resolution-pics/pics'

In [6]:
X = np.array(X)
y = np.array(y)

input = layers.Input(shape=(hr_size, hr_size, 3))
x = layers.Conv2D(32, (9, 9), padding="same", activation="relu", name="level1")(input)
x = layers.Conv2D(32, (5, 5), padding='same', activation="relu", name="level2")(x)
out = layers.Conv2D(3, (3, 3), padding='same', activation="relu", name="out")(x)

model = keras.Model(input, out)

model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])

In [7]:
X = X.astype('float32')
X /= 255.0
y = y.astype('float32')
y /= 255.0
X.shape()

In [8]:
model.fit(X, y, epochs=100, validation_split = 0.2, batch_size = 10)

Train on 8000 samples, validate on 2000 samples
Epoch 1/100
8000/8000 [==============================] - 9s 1ms/sample - loss: 0.0039 - acc: 0.8322 - val_loss: 0.0017 - val_acc: 0.8616
Epoch 2/100
8000/8000 [==============================] - 7s 905us/sample - loss: 0.0020 - acc: 0.8763 - val_loss: 0.0015 - val_acc: 0.8906
Epoch 3/100
8000/8000 [==============================] - 7s 898us/sample - loss: 0.0019 - acc: 0.8981 - val_loss: 0.0015 - val_acc: 0.8217
Epoch 4/100
8000/8000 [==============================] - 7s 921us/sample - loss: 0.0018 - acc: 0.8901 - val_loss: 0.0014 - val_acc: 0.9012
Epoch 5/100
8000/8000 [==============================] - 7s 895us/sample - loss: 0.0018 - acc: 0.8946 - val_loss: 0.0014 - val_acc: 0.8676
Epoch 6/100
8000/8000 [==============================] - 7s 889us/sample - loss: 0.0024 - acc: 0.8784 - val_loss: 0.0015 - val_acc: 0.8820
Epoch 7/100
8000/8000 [==============================] - 7s 888us/sample - loss: 0.0018 - acc: 0.8977 - val_loss: 0.0015

In [16]:
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model.h5")
print("Saved model to disk")


Saved model to disk


In [18]:
from IPython.display import FileLink
FileLink(r'model.h5')

/kaggle/working/model.h5